# [Day 19](https://adventofcode.com/2020/day/19): Monster Messages

In [1]:
from itertools import takewhile

with open("../data/19.txt", "r") as f:
    lines = (l.strip() for l in f.readlines())

rules, messages = list(takewhile(bool, lines)), list(lines)
rules = {n: [tuple(ns.split()) for ns in r.split(" | ")]
            for rule in rules for n, r in [rule.split(": ")]}

## Part 1

In [2]:
from functools import lru_cache
import re

@lru_cache
def regex(n):
    """Regex for rule n."""
    return "|".join(map(cat, rules[n]))

@lru_cache
def cat(rule):
    if len(rule) == 1 and (char := rule[0][1]) in {"a", "b"}:
        return char
    return f"({')('.join(map(regex, rule))})"

rule0 = re.compile(f"{regex('0')}$")

assert 187 == sum(1 for m in messages if rule0.match(m))

## Part 2

The updated rules 8 and 11 are
```
8: 42 | 42 8
11: 42 31 | 42 11 31
```
For my data, rule 0 is `8 11`. Consequently, the updated rule 0 is the infinite sum
```
0: 42 42 31 | ... | 42...42 31...31 | ...
```
where for each summand there are more `42`'s than `31`'s.

In [3]:
from itertools import accumulate, repeat

def matchone(n):
    rule = re.compile(regex(n))
    def match(msg):
        """Match rule at the beginning of a message."""
        if (m := rule.match(msg)) is None:
            return None, msg
        _, i = m.span()
        return m.group(), msg[i:]
    return match

def matchall(n):
    firstmatch = matchone(n)
    matchrule = lambda m, _: firstmatch(m[1])
    def match(msg):
        """Match rule successively from the beginning of a message."""
        matches = enumerate(accumulate(repeat(("", msg)), matchrule))
        return next((i, msg) for i, (m, msg) in matches if m is None)
    return match

match42 = matchall("42")
match31 = matchall("31")

def match0(msg):
    """Does a message match modified rule 0?"""
    n42, rest = match42(msg)
    n31, rest = match31(rest)
    return 1 < n31 < n42 and not rest

assert 392 == sum(1 for msg in messages if match0(msg))